# Import libraries

In [ ]:
import geotrellis.proj4._

import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.GeoTiff
import geotrellis.raster.io.geotiff.tags.TiffTags
import geotrellis.raster.io.geotiff.reader.GeoTiffReader

import geotrellis.spark._
import geotrellis.spark.io.hadoop._
import geotrellis.spark.io.hadoop.formats._
import geotrellis.spark.io.RasterReader
import geotrellis.spark.tiling.FloatingLayoutScheme

import geotrellis.vector._

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.Path

import geotrellis.proj4._
import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.GeoTiff
import geotrellis.raster.io.geotiff.tags.TiffTags
import geotrellis.raster.io.geotiff.reader.GeoTiffReader
import geotrellis.spark._
import geotrellis.spark.io.hadoop._
import geotrellis.spark.io.hadoop.formats._
import geotrellis.spark.io.RasterReader
import geotrellis.spark.tiling.FloatingLayoutScheme
import geotrellis.vector._
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.Path


# Important variables

In [ ]:
// implicit variable (important variables to run the functions in the geotrellis library)
implicit val sparkContext = sc

val rr = implicitly[RasterReader[HadoopGeoTiffRDD.Options, (ProjectedExtent, Tile)]]

sparkContext: org.apache.spark.SparkContext = org.apache.spark.SparkContext@49dd33f
rr: geotrellis.spark.io.RasterReader[geotrellis.spark.io.hadoop.HadoopGeoTiffRDD.Options,(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = geotrellis.spark.io.RasterReader$$anon$1@7083c9de


# Parameters

In [ ]:
val HdfsUrl = "hdfs://hupi-factory-02-01-01-01/"
val dataRepo = "user/factory02/thailand_workshop/ndvi/"
val landsatName2017 = "LC08_L1TP_125052_20171231_20180103_01_T1"
val landsatName2007 = "LT05_L1GS_125052_20070915_20161112_01_T2"

HdfsUrl: String = hdfs://hupi-factory-02-01-01-01/
dataRepo: String = user/factory02/thailand_workshop/ndvi/
landsatName2017: String = LC08_L1TP_125052_20171231_20180103_01_T1
landsatName2007: String = LT05_L1GS_125052_20070915_20161112_01_T2


In [ ]:
val saveName = HdfsUrl + dataRepo + landsatName2017 + "VS" + landsatName2007

saveName: String = hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/ndvi/LC08_L1TP_125052_20171231_20180103_01_T1VSLT05_L1GS_125052_20070915_20161112_01_T2


# To avoid duplicates in HDFS

In [ ]:
val conf = sc.hadoopConfiguration  
val fs = org.apache.hadoop.fs.FileSystem.get(new java.net.URI(HdfsUrl), conf)

conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
fs: org.apache.hadoop.fs.FileSystem = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-753263480_11, ugi=root (auth:SIMPLE)]]


In [ ]:
// Remove the image png if it's there already
fs.delete(new Path(saveName + ".png"),true)

res6: Boolean = true


true

# Compare 2 NDVI of 2 images 

To compare 2 NDVI, we compute the difference of NDVI and render to PNG

Here, we will take the NDVI in 2017 and in 2007

#### Firstly, we load NDVI in GeoTiff from HDFS 

In [ ]:
val options =
HadoopGeoTiffRDD.Options(
  numPartitions = Some(100)
)

options: geotrellis.spark.io.hadoop.HadoopGeoTiffRDD.Options = Options(List(.tif, .TIF, .tiff, .TIFF),None,TIFFTAG_DATETIME,yyyy:MM:dd HH:mm:ss,Some(256),Some(100),Some(134217728),None)


In [ ]:
val ndvi_2017 = HadoopGeoTiffRDD[ProjectedExtent, Tile](
      new Path(HdfsUrl + dataRepo + landsatName2017 + ".tif"), 
      options).map(l => (l._1, l._2.convert(DoubleConstantNoDataCellType)))

val ndvi_2007 = HadoopGeoTiffRDD[ProjectedExtent, Tile](
      new Path(HdfsUrl + dataRepo + landsatName2007 + ".tif"), 
      options).map(l => (l._1, l._2.convert(DoubleConstantNoDataCellType)))

ndvi_2017: org.apache.spark.rdd.RDD[(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = MapPartitionsRDD[4] at map at <console>:109
ndvi_2007: org.apache.spark.rdd.RDD[(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = MapPartitionsRDD[9] at map at <console>:113


In ndvi_2017, we replace all the negative value to -99. The reason that we need to make the negative value of ndvi 2017 more "negative" is if we have -0.5 - (-0.7) = 0.2 => in the output, we may see some strange result...

In [ ]:
val ndvi_2017_updated = ndvi_2017.map {
  case (pe, tile) => {
    val arrayTile = tile.toArrayDouble
    val newArrayTile = arrayTile.map {
      case (value) => {
        if (value < 0) 
        {-99.0} 
        else {value}
      }
    }
    (pe, DoubleArrayTile(newArrayTile, tile.cols, tile.rows).tile)    
  }
}

ndvi_2017_updated: org.apache.spark.rdd.RDD[(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = MapPartitionsRDD[10] at map at <console>:108


#### Then, we convert ProjectedExtent to SpatialKey 

In [ ]:
val (_, metadata_2017) = ndvi_2017_updated.collectMetadata[SpatialKey](FloatingLayoutScheme())
val tiles_2017 = ndvi_2017_updated.tileToLayout[SpatialKey](metadata_2017)

val (_, metadata_2007) = ndvi_2007.collectMetadata[SpatialKey](FloatingLayoutScheme())
val tiles_2007 = ndvi_2007.tileToLayout[SpatialKey](metadata_2007)

metadata_2017: geotrellis.spark.TileLayerMetadata[geotrellis.spark.SpatialKey] = TileLayerMetadata(float64,GridExtent(Extent(528585.0, 1156335.0, 758985.0, 1394415.0),30.0,30.0),Extent(528585.0, 1156335.0, 758985.0, 1394415.0),EPSG:32648,KeyBounds(SpatialKey(0,0),SpatialKey(29,30)))
tiles_2017: org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.raster.Tile)] = ShuffledRDD[13] at reduceByKey at TileRDDMerge.scala:51
metadata_2007: geotrellis.spark.TileLayerMetadata[geotrellis.spark.SpatialKey] = TileLayerMetadata(float64,GridExtent(Extent(527085.0, 1167075.0, 765165.0, 1382115.0),30.0,30.0),Extent(527085.0, 1167075.0, 765165.0, 1382115.0),EPSG:32648,KeyBounds(SpatialKey(0,0),SpatialKey(30,27)))
tiles_2007: org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.r...

#### And compute the difference of NDVI

In [ ]:
val difference_ndvi = (tiles_2017 - tiles_2007)

difference_ndvi: org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.raster.Tile)] = MapPartitionsRDD[20] at mapValues at CombineMethods.scala:32


#### FInally create a raster by stitching all RDD with metadata

In [ ]:
val raster = ContextRDD(difference_ndvi, metadata_2017).stitch

raster: geotrellis.raster.Raster[geotrellis.raster.Tile] = Raster(DoubleConstantNoDataArrayTile([D@5ce53313,7680,7168),Extent(528585.0, 1179375.0, 758985.0, 1394415.0))


# By visualization

We create a PNG of difference_ndvi and save it in HDFS and in server. Here the colorMap, we use < - 2 and make it to blue color (it just means that this is a zone without green in 2007 and 2017). If we see some green in the photo PNG, it means that this zone is greener in 2017 compared to 2007. If it's yellow => it's less green than before.

#### We create the color map

In [ ]:
// NDVI goes from -1 to 1, so <= 0 => ffffe5ff; (0;0.1] : f7fcb9ff, etc.
val ndviColormap = "-2:cc7a00ff;0:ffffe5ff;0.1:f7fcb9ff;0.2:d9f0a3ff;0.3:addd8eff;0.4:78c679ff;0.5:41ab5dff;0.6:238443ff;0.7:006837ff;1:004529ff;2:004529ff"

// Get color map from the application.conf settings file.
val colorMap = ColorMap.fromStringDouble(ndviColormap).get

ndviColormap: String = -2:cc7a00ff;0:ffffe5ff;0.1:f7fcb9ff;0.2:d9f0a3ff;0.3:addd8eff;0.4:78c679ff;0.5:41ab5dff;0.6:238443ff;0.7:006837ff;1:004529ff;2:004529ff
colorMap: geotrellis.raster.render.ColorMap = geotrellis.raster.render.DoubleColorMap@5b0eb0ba


#### And render tile to PNG with colorMap 

In [ ]:
val image_png = raster.tile.renderPng(colorMap)

image_png: geotrellis.raster.render.Png = Png([B@14e768b)


#### Save to HDFS 

In [ ]:
image_png.write(new Path(saveName + ".png"))

No codec found for hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/ndvi/LC08_L1TP_125052_20171231_20180103_01_T1VSLT05_L1GS_125052_20070915_20161112_01_T2.png, writing without compression.


# Save to server and print image in notebook

In [ ]:
val ndviPath_server = "/opt/docker/notebooks/data/" + landsatName2017 + "VS" + landsatName2007 + ".png"

ndviPath_server: String = /opt/docker/notebooks/data/LC08_L1TP_125052_20171231_20180103_01_T1VSLT05_L1GS_125052_20070915_20161112_01_T2.png


In [ ]:
image_png.write(ndviPath_server)

In [ ]:
import java.io.File

import java.io.File


In [ ]:
val image_ndvi = img() // default type and size
image_ndvi.file(new File(ndviPath_server))

image_ndvi: notebook.front.SingleConnectedWidget[java.awt.image.BufferedImage]{implicit val codec: notebook.Codec[play.api.libs.json.JsValue,java.awt.image.BufferedImage]; lazy val toHtml: scala.xml.Elem; def url(u: java.net.URL): Unit; def file(f: java.io.File): Unit} = <$anon$1 widget>


In [ ]:
image_ndvi

res39: notebook.front.SingleConnectedWidget[java.awt.image.BufferedImage]{implicit val codec: notebook.Codec[play.api.libs.json.JsValue,java.awt.image.BufferedImage]; lazy val toHtml: scala.xml.Elem; def url(u: java.net.URL): Unit; def file(f: java.io.File): Unit} = <$anon$1 widget>
